In [83]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import hstack,vstack
from scipy import sparse
import lightgbm
from sklearn.model_selection import train_test_split 
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text  import CountVectorizer
import starterkit
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
import os
import io
from bayes_opt import BayesianOptimization

# Loading Data

In [4]:
train_data = pd.read_csv('D:\\Machine Learning\\Datasets\\EmoContext\\train.txt',
                        sep='\t', index_col='id')
X = train_data.iloc[:,:-1]
y = train_data.iloc[:,-1]
le = LabelEncoder()
y = le.fit_transform(y)

In [5]:
dev_data = pd.read_csv('D:\\Machine Learning\\Datasets\\EmoContext\\dev.txt',
                        sep='\t', index_col='id')
X_dev = dev_data.iloc[:,:-1]
y_dev = dev_data.iloc[:,-1]
y_dev = le.transform(y_dev)

In [6]:
X_test = pd.read_csv('D:\\Machine Learning\\Datasets\\EmoContext\\testwithoutlabels.txt',
                        sep='\t', index_col='id')

In [13]:
features_path = 'D:\\Machine Learning\\Datasets\\EmoContext\\Features\\FeatSelection\\' 
X_log_select_train = sparse.load_npz(features_path + r'\log_reg\train.npz')
X_log_select_dev = sparse.load_npz(features_path + r'\log_reg\dev.npz')
X_log_select_test = sparse.load_npz(features_path + r'\log_reg\test.npz')

X_lgbm_feats_train = sparse.load_npz(features_path + r'\lgbm\train.npz')
X_lgbm_feats_dev = sparse.load_npz(features_path + r'\lgbm\dev.npz')
X_lgbm_feats_test = sparse.load_npz(features_path + r'\lgbm\test.npz')

X_chi2_train = sparse.load_npz(features_path + r'\chi2\train.npz')
X_chi2_dev = sparse.load_npz(features_path + r'\chi2\dev.npz')
X_chi2_test = sparse.load_npz(features_path + r'\chi2\test.npz')

In [8]:
def load_feats(feats_dir, folder):
    dict_to_feed = {}
    full_path = os.path.join(feats_dir, folder)
    files = [f for f in os.listdir(full_path) 
             if os.path.isfile(os.path.join(full_path, f)) and f[-4:] =='.npz']
    
    for f in files:
        dict_to_feed[f[:-4]] = sparse.load_npz(os.path.join(full_path, f))
    return dict_to_feed

In [9]:
feats_path = 'D:\\Machine Learning\\Datasets\\EmoContext\\Features'

train_feats_dict = load_feats(feats_path, 'train')
dev_feats_dict = load_feats(feats_path, 'dev')
test_feats_dict = load_feats(feats_path, 'test')

train_feats = hstack(list(train_feats_dict.values()))
dev_feats = hstack(list(dev_feats_dict.values()))
test_feats = hstack(list(test_feats_dict.values()))

In [36]:
params =  {'num_leaves': 45,
                                        'feature_fraction':  0.9,
                                        'bagging_fraction': 1,
                                        'max_depth': 8,
                                        'lambda_l1':  5,
                                        'lambda_l2':  3,
                                        'min_split_gain':  0.1,
                                        'min_child_weight': 50}
clf = lightgbm.LGBMClassifier(**params, random_state=42)
clf.fit(X_log_select_train, y)

LGBMClassifier(bagging_fraction=1, boosting_type='gbdt', class_weight=None,
        colsample_bytree=1.0, feature_fraction=0.9,
        importance_type='split', lambda_l1=5, lambda_l2=3,
        learning_rate=0.1, max_depth=8, min_child_samples=20,
        min_child_weight=50, min_split_gain=0.1, n_estimators=100,
        n_jobs=-1, num_leaves=45, objective=None, random_state=42,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)

(30160,)

In [56]:
def lightGbmEval(X_train, y_train, X_dev, y_dev):
    def lightGbmTrainDevKitScore(num_leaves, feature_fraction, bagging_fraction,
                 max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
        params = {'application':'multiclass',
                  'num_iterations':4000, 'learning_rate':0.05,
                 # 'early_stopping_round':100, 
                  'metric':'multi_logloss'}
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        #lgb_train = lgb.Dataset(X_train, y_train)
        #lgb_eval = lgb.Dataset(X_dev, y_dev, reference=lgb_train)
        clf = lightgbm.LGBMClassifier(**params, random_state=42)
        clf.fit(X_train, y_train)

        lb = LabelBinarizer()
        lb.fit(y_train)

        dev_pred = clf.predict(X_dev)
        ground = lb.transform(y_dev)
        predictions = lb.transform(dev_pred)
        return starterkit.getMetrics(predictions=predictions, ground=ground)[-1]
    return lightGbmTrainDevKitScore

In [57]:
lgb_eval = lightGbmEval(X_log_select_train, y, X_log_select_dev, y_dev)

In [58]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (24, 45),
                                        'feature_fraction': (0.1, 0.9),
                                        'bagging_fraction': (0.8, 1),
                                        'max_depth': (5, 9),
                                        'lambda_l1': (0, 5),
                                        'lambda_l2': (0, 3),
                                        'min_split_gain': (0.001, 0.1),
                                        'min_child_weight': (5, 50)}, random_state=0)
lgbBO.maximize(init_points=20, n_iter=40)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------
True Positives per class :  [ 116.  101. 2124.   91.]
False Positives per class :  [94. 80. 94. 55.]
False Negatives per class :  [ 34.  41. 214.  34.]
Class happy : Precision : 0.558, Recall : 0.711, F1 : 0.625
Class sad : Precision : 0.958, Recall : 0.908, F1 : 0.932
Class angry : Precision : 0.623, Recall : 0.728, F1 : 0.672
Ignoring the Others class, Macro Precision : 0.7130, Macro Recall : 0.7826, Macro F1 : 0.7462
Ignoring the Others class, Micro TP : 2316, FP : 229, FN : 289
Accuracy : 0.8828, Micro Precision : 0.9100, Micro Recall : 0.8891, Micro F1 : 0.8994
|  1        |  0.8994   |  0.9098   |  0.6722   |  3.014    |  1.635    |  6.695    |  34.07    |  0.04432  |  42.73    |


C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


True Positives per class :  [ 113.  101. 2114.   88.]
False Positives per class :  [100.  76.  96.  67.]
False Negatives per class :  [ 37.  41. 224.  37.]
Class happy : Precision : 0.571, Recall : 0.711, F1 : 0.633
Class sad : Precision : 0.957, Recall : 0.904, F1 : 0.930
Class angry : Precision : 0.568, Recall : 0.704, F1 : 0.629
Ignoring the Others class, Macro Precision : 0.6983, Macro Recall : 0.7732, Macro F1 : 0.7338
Ignoring the Others class, Micro TP : 2303, FP : 239, FN : 302
Accuracy : 0.8770, Micro Precision : 0.9060, Micro Recall : 0.8841, Micro F1 : 0.8949
|  2        |  0.8949   |  0.9927   |  0.4068   |  3.959    |  1.587    |  7.272    |  46.65    |  0.008033 |  25.83    |


C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


True Positives per class :  [ 113.  101. 2111.   89.]
False Positives per class :  [104.  80.  96.  61.]
False Negatives per class :  [ 37.  41. 227.  36.]
Class happy : Precision : 0.558, Recall : 0.711, F1 : 0.625
Class sad : Precision : 0.957, Recall : 0.903, F1 : 0.929
Class angry : Precision : 0.593, Recall : 0.712, F1 : 0.647
Ignoring the Others class, Macro Precision : 0.7026, Macro Recall : 0.7754, Macro F1 : 0.7372
Ignoring the Others class, Micro TP : 2301, FP : 237, FN : 304
Accuracy : 0.8762, Micro Precision : 0.9066, Micro Recall : 0.8833, Micro F1 : 0.8948
|  3        |  0.8948   |  0.804    |  0.7661   |  3.891    |  2.61     |  8.914    |  40.96    |  0.04669  |  40.39    |


...

True Positives per class :  [ 120.  104. 2152.   93.]
False Positives per class :  [83. 75. 82. 46.]
False Negatives per class :  [ 30.  38. 186.  32.]
Class happy : Precision : 0.581, Recall : 0.732, F1 : 0.648
Class sad : Precision : 0.963, Recall : 0.920, F1 : 0.941
Class angry : Precision : 0.669, Recall : 0.744, F1 : 0.705
Ignoring the Others class, Macro Precision : 0.7378, Macro Recall : 0.7989, Macro F1 : 0.7672
Ignoring the Others class, Micro TP : 2349, FP : 203, FN : 256
Accuracy : 0.8962, Micro Precision : 0.9205, Micro Recall : 0.9017, Micro F1 : 0.9110
|  58       |  0.911    |  0.808    |  0.1884   |  0.00801  |  0.4301   |  8.666    |  5.798    |  0.06368  |  39.3     |


C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


True Positives per class :  [ 120.  105. 2146.   96.]
False Positives per class :  [89. 75. 81. 43.]
False Negatives per class :  [ 30.  37. 192.  29.]
Class happy : Precision : 0.583, Recall : 0.739, F1 : 0.652
Class sad : Precision : 0.964, Recall : 0.918, F1 : 0.940
Class angry : Precision : 0.691, Recall : 0.768, F1 : 0.727
Ignoring the Others class, Macro Precision : 0.7459, Macro Recall : 0.8084, Macro F1 : 0.7759
Ignoring the Others class, Micro TP : 2347, FP : 199, FN : 258
Accuracy : 0.8955, Micro Precision : 0.9218, Micro Recall : 0.9010, Micro F1 : 0.9113
|  59       |  0.9113   |  0.8554   |  0.1405   |  0.06109  |  0.1031   |  7.18     |  5.243    |  0.09259  |  24.69    |


C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


True Positives per class :  [ 118.  102. 2153.   95.]
False Positives per class :  [88. 72. 83. 44.]
False Negatives per class :  [ 32.  40. 185.  30.]
Class happy : Precision : 0.586, Recall : 0.718, F1 : 0.646
Class sad : Precision : 0.963, Recall : 0.921, F1 : 0.941
Class angry : Precision : 0.683, Recall : 0.760, F1 : 0.720
Ignoring the Others class, Macro Precision : 0.7442, Macro Recall : 0.7997, Macro F1 : 0.7710
Ignoring the Others class, Micro TP : 2350, FP : 199, FN : 255
Accuracy : 0.8958, Micro Precision : 0.9219, Micro Recall : 0.9021, Micro F1 : 0.9119
|  60       |  0.9119   |  0.9855   |  0.1109   |  0.08916  |  0.295    |  7.8      |  5.12     |  0.05835  |  32.69    |


In [62]:
target = -1
max_params = {}
for res in lgbBO.res:
    if target < res['target']:
        target = res['target']
        max_params = res['params']

In [64]:
print(target)
max_params

0.9133553014150029


{'bagging_fraction': 0.9819566720743442,
 'feature_fraction': 0.16775915680389533,
 'lambda_l1': 0.5070887211533209,
 'lambda_l2': 1.200993517318602,
 'max_depth': 8.953397047989986,
 'min_child_weight': 5.04261361744641,
 'min_split_gain': 0.07362854888667725,
 'num_leaves': 34.19768540650169}

In [65]:
lightGbmEval(X_log_select_train, y, X_log_select_dev, y_dev)(**max_params)

C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


True Positives per class :  [ 119.  105. 2156.   95.]
False Positives per class :  [82. 72. 82. 44.]
False Negatives per class :  [ 31.  37. 182.  30.]
Class happy : Precision : 0.593, Recall : 0.739, F1 : 0.658
Class sad : Precision : 0.963, Recall : 0.922, F1 : 0.942
Class angry : Precision : 0.683, Recall : 0.760, F1 : 0.720
Ignoring the Others class, Macro Precision : 0.7467, Macro Recall : 0.8072, Macro F1 : 0.7758
Ignoring the Others class, Micro TP : 2356, FP : 198, FN : 249
Accuracy : 0.8984, Micro Precision : 0.9225, Micro Recall : 0.9044, Micro F1 : 0.9134


0.9133553014150029

In [70]:
def lightgbmClassifier(num_leaves, feature_fraction, bagging_fraction,
                 max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
        params = {'application':'multiclass',
                  'num_iterations':4000, 'learning_rate':0.05,
                 # 'early_stopping_round':100, 
                  'metric':'multi_logloss'}
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        return lightgbm.LGBMClassifier(**params, random_state=42)

In [71]:
clf = lightgbmClassifier(**max_params)
clf.fit(X_log_select_train, y)

C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMClassifier(application='multiclass', bagging_fraction=0.9819566720743442,
        boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        feature_fraction=0.16775915680389533, importance_type='split',
        lambda_l1=0.5070887211533209, lambda_l2=1.200993517318602,
        learning_rate=0.05, max_depth=9, metric='multi_logloss',
        min_child_samples=20, min_child_weight=5.04261361744641,
        min_split_gain=0.07362854888667725, n_estimators=100, n_jobs=-1,
        num_iterations=4000, num_leaves=34, objective=None,
        random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [72]:
test_pred = clf.predict(X_log_select_test)

In [74]:
def save_test_set(test_pred):
    test_labels = le.inverse_transform(test_pred)
    solutionPath = 'D:\\Machine Learning\\Datasets\\EmoContext\\test.txt'
    testDataPath = 'D:\\Machine Learning\\Datasets\\EmoContext\\testwithoutlabels.txt'
    with io.open(solutionPath, "w", encoding="utf8") as fout:
            fout.write('\t'.join(["id", "turn1", "turn2", "turn3", "label"]) + '\n')        
            with io.open(testDataPath, encoding="utf8") as fin:
                fin.readline()
                for lineNum, line in enumerate(fin):
                    fout.write('\t'.join(line.strip().split('\t')[:4]) + '\t')
                    fout.write(test_labels[lineNum] + '\n')

In [75]:
save_test_set(test_pred)

In [76]:
clf = lightgbmClassifier(**max_params)
clf.fit(X_lgbm_feats_train, y)

C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMClassifier(application='multiclass', bagging_fraction=0.9819566720743442,
        boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        feature_fraction=0.16775915680389533, importance_type='split',
        lambda_l1=0.5070887211533209, lambda_l2=1.200993517318602,
        learning_rate=0.05, max_depth=9, metric='multi_logloss',
        min_child_samples=20, min_child_weight=5.04261361744641,
        min_split_gain=0.07362854888667725, n_estimators=100, n_jobs=-1,
        num_iterations=4000, num_leaves=34, objective=None,
        random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [77]:
test_pred = clf.predict(X_lgbm_feats_test)
save_test_set(test_pred)

C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\basic.py:452: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [78]:
lgb_eval = lightGbmEval(X_lgbm_feats_train, y, X_lgbm_feats_dev, y_dev)
lgbBO_lgbm = BayesianOptimization(lgb_eval, {'num_leaves': (24, 45),
                                        'feature_fraction': (0.1, 0.9),
                                        'bagging_fraction': (0.8, 1),
                                        'max_depth': (5, 9),
                                        'lambda_l1': (0, 5),
                                        'lambda_l2': (0, 3),
                                        'min_split_gain': (0.001, 0.1),
                                        'min_child_weight': (5, 50)}, random_state=0)
lgbBO_lgbm.maximize(init_points=20, n_iter=40)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------


C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\basic.py:452: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


True Positives per class :  [ 114.  103. 2117.   89.]
False Positives per class :  [93. 77. 95. 67.]
False Negatives per class :  [ 36.  39. 221.  36.]
Class happy : Precision : 0.572, Recall : 0.725, F1 : 0.640
Class sad : Precision : 0.957, Recall : 0.905, F1 : 0.931
Class angry : Precision : 0.571, Recall : 0.712, F1 : 0.633
Ignoring the Others class, Macro Precision : 0.6999, Macro Recall : 0.7809, Macro F1 : 0.7382
Ignoring the Others class, Micro TP : 2309, FP : 239, FN : 296
Accuracy : 0.8795, Micro Precision : 0.9062, Micro Recall : 0.8864, Micro F1 : 0.8962
|  1        |  0.8962   |  0.9098   |  0.6722   |  3.014    |  1.635    |  6.695    |  34.07    |  0.04432  |  42.73    |


C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\basic.py:452: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


True Positives per class :  [ 112.  100. 2114.   88.]
False Positives per class :  [105.  78.  98.  60.]
False Negatives per class :  [ 38.  42. 224.  37.]
Class happy : Precision : 0.562, Recall : 0.704, F1 : 0.625
Class sad : Precision : 0.956, Recall : 0.904, F1 : 0.929
Class angry : Precision : 0.595, Recall : 0.704, F1 : 0.645
Ignoring the Others class, Macro Precision : 0.7040, Macro Recall : 0.7708, Macro F1 : 0.7359
Ignoring the Others class, Micro TP : 2302, FP : 236, FN : 303
Accuracy : 0.8762, Micro Precision : 0.9070, Micro Recall : 0.8837, Micro F1 : 0.8952
|  2        |  0.8952   |  0.9927   |  0.4068   |  3.959    |  1.587    |  7.272    |  46.65    |  0.008033 |  25.83    |


...

True Positives per class :  [ 116.   99. 2149.   92.]
False Positives per class :  [86. 72. 95. 46.]
False Negatives per class :  [ 34.  43. 189.  33.]
Class happy : Precision : 0.579, Recall : 0.697, F1 : 0.633
Class sad : Precision : 0.958, Recall : 0.919, F1 : 0.938
Class angry : Precision : 0.667, Recall : 0.736, F1 : 0.700
Ignoring the Others class, Macro Precision : 0.7344, Macro Recall : 0.7841, Macro F1 : 0.7585
Ignoring the Others class, Micro TP : 2340, FP : 213, FN : 265
Accuracy : 0.8915, Micro Precision : 0.9166, Micro Recall : 0.8983, Micro F1 : 0.9073
|  58       |  0.9073   |  0.9965   |  0.1311   |  3.574    |  0.1798   |  8.857    |  5.283    |  0.07913  |  40.77    |


C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\basic.py:452: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


True Positives per class :  [ 118.  101. 2153.   92.]
False Positives per class :  [85. 72. 91. 43.]
False Negatives per class :  [ 32.  41. 185.  33.]
Class happy : Precision : 0.584, Recall : 0.711, F1 : 0.641
Class sad : Precision : 0.959, Recall : 0.921, F1 : 0.940
Class angry : Precision : 0.681, Recall : 0.736, F1 : 0.708
Ignoring the Others class, Macro Precision : 0.7416, Macro Recall : 0.7894, Macro F1 : 0.7647
Ignoring the Others class, Micro TP : 2346, FP : 206, FN : 259
Accuracy : 0.8944, Micro Precision : 0.9193, Micro Recall : 0.9006, Micro F1 : 0.9098
|  59       |  0.9098   |  0.9898   |  0.121    |  0.5539   |  0.5947   |  8.836    |  12.18    |  0.08969  |  42.49    |


C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\basic.py:452: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


True Positives per class :  [ 116.  102. 2142.   97.]
False Positives per class :  [91. 75. 86. 46.]
False Negatives per class :  [ 34.  40. 196.  28.]
Class happy : Precision : 0.576, Recall : 0.718, F1 : 0.639
Class sad : Precision : 0.961, Recall : 0.916, F1 : 0.938
Class angry : Precision : 0.678, Recall : 0.776, F1 : 0.724
Ignoring the Others class, Macro Precision : 0.7387, Macro Recall : 0.8035, Macro F1 : 0.7697
Ignoring the Others class, Micro TP : 2341, FP : 207, FN : 264
Accuracy : 0.8918, Micro Precision : 0.9188, Micro Recall : 0.8987, Micro F1 : 0.9086
|  60       |  0.9086   |  0.9941   |  0.117    |  0.03446  |  1.266    |  8.847    |  6.855    |  0.004214 |  39.45    |


In [79]:
target = -1
max_params_lgbm = {}
for res in lgbBO_lgbm.res:
    if target < res['target']:
        target = res['target']
        max_params_lgbm = res['params']

In [81]:
target

0.9141971721867131

In [80]:
max_params_lgbm

{'bagging_fraction': 0.9868468179754804,
 'feature_fraction': 0.1087286871299523,
 'lambda_l1': 0.21001373440221238,
 'lambda_l2': 1.6076127185605533,
 'max_depth': 8.709472188506037,
 'min_child_weight': 6.411112138659305,
 'min_split_gain': 0.09949353868448442,
 'num_leaves': 38.75237749966753}

In [89]:
y_dev.shape

(2755,)

(32915,)

In [114]:
clf_lgbm = lightgbmClassifier(**max_params_lgbm)
clf_lgbm.fit(vstack([X_lgbm_feats_train,X_lgbm_feats_dev]), np.hstack([y, y_dev]))

C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMClassifier(application='multiclass', bagging_fraction=0.9868468179754804,
        boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        feature_fraction=0.1087286871299523, importance_type='split',
        lambda_l1=0.21001373440221238, lambda_l2=1.6076127185605533,
        learning_rate=0.05, max_depth=9, metric='multi_logloss',
        min_child_samples=20, min_child_weight=6.411112138659305,
        min_split_gain=0.09949353868448442, n_estimators=100, n_jobs=-1,
        num_iterations=4000, num_leaves=39, objective=None,
        random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [103]:
test_pred = clf_lgbm.predict(X_lgbm_feats_test)
save_test_set(test_pred)

C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\basic.py:452: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [115]:
tr_dev_pred_lgbm = clf_lgbm.predict_proba(vstack([X_lgbm_feats_train,X_lgbm_feats_dev]))
test_pred_lgbm = clf_lgbm.predict_proba(X_lgbm_feats_test)

C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\basic.py:452: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [106]:
clf = lightgbmClassifier(**max_params)
clf.fit(vstack([X_log_select_train,X_log_select_dev]), np.hstack([y, y_dev]))

C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMClassifier(application='multiclass', bagging_fraction=0.9819566720743442,
        boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        feature_fraction=0.16775915680389533, importance_type='split',
        lambda_l1=0.5070887211533209, lambda_l2=1.200993517318602,
        learning_rate=0.05, max_depth=9, metric='multi_logloss',
        min_child_samples=20, min_child_weight=5.04261361744641,
        min_split_gain=0.07362854888667725, n_estimators=100, n_jobs=-1,
        num_iterations=4000, num_leaves=34, objective=None,
        random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [116]:
tr_dev_pred_log = clf.predict_proba(vstack([X_log_select_train,X_log_select_dev]))
test_pred_log = clf.predict_proba(X_log_select_test)

In [123]:
clf_chi = lightgbmClassifier(**max_params)
clf_chi.fit(vstack([X_chi2_train,X_chi2_dev]), np.hstack([y, y_dev]))

LGBMClassifier(application='multiclass', bagging_fraction=0.9819566720743442,
        boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        feature_fraction=0.16775915680389533, importance_type='split',
        lambda_l1=0.5070887211533209, lambda_l2=1.200993517318602,
        learning_rate=0.05, max_depth=9, metric='multi_logloss',
        min_child_samples=20, min_child_weight=5.04261361744641,
        min_split_gain=0.07362854888667725, n_estimators=100, n_jobs=-1,
        num_iterations=4000, num_leaves=34, objective=None,
        random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [119]:
tr_dev_pred_chi = clf_chi.predict_proba(vstack([X_chi2_train,X_chi2_dev]))
test_pred_chi = clf_chi.predict_proba(X_chi2_test)

C:\Users\Simeon\Anaconda3\envs\First\lib\site-packages\lightgbm\basic.py:452: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [121]:
np.hstack([tr_dev_pred_lgbm,tr_dev_pred_log,tr_dev_pred_chi]).shape

(32915, 12)

In [126]:
clf_stack1 = lightgbm.LGBMClassifier()
clf_stack1.fit(np.hstack([tr_dev_pred_lgbm,tr_dev_pred_log,tr_dev_pred_chi]), np.hstack([y, y_dev]))

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [127]:
test_pred_stack = clf_stack1.predict(np.hstack([test_pred_lgbm,test_pred_log,test_pred_chi]))
save_test_set(test_pred_stack)